In [7]:
import pickle
import pandas as pd
import psycopg2 as pg
import geopandas as gpd

from Data_getter.get_data import get_buildings, get_service
from Blocks_getter.blocks_getter import BlocksModel
from Provision_getter.provision_getter import ProvisionModel

# with open('output_data/balanced_result_block.pkl', 'rb') as f:
#     updated_block = pickle.load(f)

# GROUPS = ['schools', 'kindergartens', 'recreational_areas']

city_crs = 32636
city_id = 1
service_type = "schools"
matrix = pd.read_pickle("/Users/mvin/Desktop/mp_dev/MASTERPLAN/Data/MMG_SPB.pkl")
engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
# buildings_gdf = get_buildings(engine=engine, city_crs=city_crs, city_id=city_id)
service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)
# blocks_gdf = BlocksModel(city_name='Санкт-Петербург', city_admin_level=5, city_crs=city_crs).get_blocks()
buildings_gdf = gpd.read_parquet('./output_data/buildings.parquet')
blocks_gdf = gpd.read_parquet('./output_data/blocks.parquet')
services_accessabilities = {"schools": 30, "kindergartens": 30, "recreational_areas": 30}

service_name = "schools"
model = ProvisionModel(
    SUB_GROUP=service_name,
    blocks=blocks_gdf,
    matrix=matrix,
    buildings=buildings_gdf,
    ACCS_TIME=services_accessabilities[service_name],
    service_gdf=service_gdf
)
gdf = model.run()
gdf.head()


/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/605 [00:00<?, ?it/s]

количество кварталов c сервисом schools: 51
количество жилых кварталов: 2110
количество кварталов всего: 5522
количество кварталов c ошибкой: 0


,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools,population
0,0,"POLYGON ((316983.653 6644038.696, 316986.485 6...",0,0,0,0,0
1,1,"POLYGON ((315287.843 6646411.375, 315288.850 6...",0,0,0,0,0
2,2,"POLYGON ((315582.670 6646809.216, 315582.581 6...",0,0,0,0,0
3,3,"POLYGON ((315264.235 6646596.731, 315274.102 6...",0,0,0,0,0
4,4,"POLYGON ((313479.271 6646628.746, 313479.599 6...",0,0,0,0,0


In [9]:
gdf.loc[gdf[f'provision_schools'] == 100]

,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools,population
7,7,"POLYGON ((316342.742 6646997.500, 316342.477 6...",100,196,267,0,267
67,67,"POLYGON ((316273.824 6677499.650, 316273.923 6...",100,83,71,0,71
70,70,"POLYGON ((316672.882 6677416.844, 316672.894 6...",100,83,42,0,42
75,75,"POLYGON ((307306.193 6678165.589, 307306.274 6...",100,107,53,0,53
76,76,"POLYGON ((316860.021 6677638.139, 316860.091 6...",100,83,44,0,44
...,...,...,...,...,...,...,...
6130,6130,"POLYGON ((366647.529 6629696.164, 366647.466 6...",100,5919,592,0,592
6152,6152,"POLYGON ((366083.921 6630169.358, 366084.061 6...",100,5910,873,0,873
6155,6155,"POLYGON ((366397.403 6631035.358, 366397.548 6...",100,5910,896,0,896
6157,6157,"POLYGON ((365934.781 6630293.610, 365934.499 6...",100,5910,260,0,260


In [10]:
gdf.loc[((gdf[f'population_unprov_schools'] > 0) &(gdf[f'provision_schools'] > 0))]

,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools,population
92,92,"POLYGON ((306717.568 6678136.612, 306717.770 6...",33,107,147,291,439
571,571,"POLYGON ((320624.168 6654912.212, 320624.085 6...",31,327,255,555,811
782,782,"POLYGON ((324402.745 6643448.134, 325085.481 6...",81,782,6733,1566,8300
922,922,"POLYGON ((327756.186 6640247.654, 327756.266 6...",62,935,9166,5438,14605
1357,1357,"POLYGON ((335871.591 6624196.553, 335871.662 6...",55,1357,5016,3961,8978
...,...,...,...,...,...,...,...
5755,5755,"POLYGON ((358980.149 6648268.974, 358980.063 6...",87,5755,12125,1782,13907
5757,5757,"POLYGON ((360238.160 6648864.747, 359662.114 6...",80,5757,10000,2394,12394
5916,5916,"POLYGON ((364484.991 6632347.945, 364404.695 6...",29,5916,1466,3550,5017
5925,5925,"POLYGON ((364022.684 6633357.696, 364022.750 6...",57,5925,6716,5066,11783
